## **Ingeniería y preprocesamiento de Datos**

En este notebook se desarrolla el proceso completo de preprocesamiento y limpieza de datos aplicado a un conjunto de datos del mercado de autos usados en Estados Unidos, con el objetivo de preparar información de alta calidad para modelos de predicción de precios.

El trabajo se enfoca en la reducción de ruido y complejidad del dataset, comenzando por la eliminación de variables con bajo poder predictivo o información redundante. Posteriormente, se aborda el tratamiento de valores nulos, analizando su impacto en variables clave como condición del vehículo, cilindros y sistema de tracción, y aplicando estrategias consistentes con la lógica del mercado automotriz.

Como primer filtro, se realiza la detección y eliminación de valores atípicos, considerando rangos realistas de precio, año y kilometraje, con el fin de evitar que observaciones extremas distorsionen el entrenamiento de los modelos. Este paso es especialmente relevante dada la gran magnitud del dataset y la alta variabilidad inherente a los precios de los vehículos usados.

Adicionalmente, se normalizan y agrupan categorías con inconsistencias semánticas (por ejemplo, errores de escritura o categorías poco representativas), reduciendo la dimensionalidad y mejorando la estabilidad de los modelos posteriores. El conjunto de datos resultante es almacenado en un formato eficiente y optimizado para análisis y modelado, permitiendo un flujo de trabajo más ágil y reproducible.

Este notebook sienta las bases para las siguientes etapas del proyecto, donde se comparan distintos algoritmos de regresión lineal, modelos basados en árboles y redes neuronales, asegurando que todos los modelos trabajen sobre datos limpios, coherentes y representativos del mercado real.

In [ ]:
import pandas as pd
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
!unzip /content/drive/MyDrive/dataset/criaglist.zip

Archive:  /content/drive/MyDrive/dataset/criaglist.zip
  inflating: vehicles.csv            


In [ ]:
df = pd.read_csv('/content/vehicles.csv')

In [ ]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [ ]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [ ]:
drops = ['id','url','region','region_url','image_url','description','county','lat','long','posting_date','VIN','state','size']
df.drop(drops,axis=1,inplace=True)

In [ ]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como parte del proceso de limpieza inicial, se eliminaron múltiples variables que no aportan valor predictivo al problema de estimación del precio o que no son utilizables en un contexto real de modelado. Entre estas se encuentran identificadores únicos, URLs, información geográfica altamente granular, campos textuales no estructurados y metadatos propios de la publicación del anuncio.

## **Tratamiento de valores nulos variables continuas**

In [ ]:
df.select_dtypes(include=['float','int']).isnull().mean()

,0
price,0.000000
year,0.002823
odometer,0.010307


In [ ]:
df.year.median()

2013.0

In [ ]:
df.odometer.median()

85548.0

In [ ]:
df.year = df.year.fillna(df.year.median())
df.odometer = df.odometer.fillna(df.odometer.median())

Las variables numéricas clave como year y odometer presentan una baja proporción de valores faltantes y tienen una relación directa con el precio del vehículo. Dado que se trata de atributos fundamentales y con distribuciones que pueden presentar asimetrías y valores atípicos, se optó por imputar los valores nulos utilizando la mediana.

La mediana es una medida robusta que no se ve afectada por valores extremos, lo que la convierte en una opción adecuada en el contexto del mercado de autos usados, donde pueden existir registros con kilometrajes inusualmente altos o inconsistencias en el año del vehículo.

Esta estrategia permite preservar la información relevante, mantener la escala original de las variables y evitar distorsiones en el entrenamiento de los modelos predictivos, garantizando al mismo tiempo estabilidad y consistencia en los datos.

In [ ]:
df.describe()

,price,year,odometer
count,4.268800e+05,426880.000000,4.268800e+05
mean,7.519903e+04,2011.240173,9.791454e+04
std,1.218228e+07,9.439234,2.127801e+05
min,0.000000e+00,1900.000000,0.000000e+00
25%,5.900000e+03,2008.000000,3.813000e+04
50%,1.395000e+04,2013.000000,8.554800e+04
75%,2.648575e+04,2017.000000,1.330000e+05
max,3.736929e+09,2022.000000,1.000000e+07


Una vez imputados los valores nulos en las variables numéricas continuas, se realizó un análisis de consistencia semántica de los datos. Durante este proceso se identificaron registros con valores irreales o inconsistentes que no tienen sentido dentro del paradigma de la venta de autos usados.

En particular, se detectaron:

* Precios mínimos o cercanos a cero, los cuales no representan transacciones reales y suelen corresponder a errores de captura o anuncios incompletos.

* Valores de kilometraje extremadamente bajos o iguales a cero, lo cual es poco plausible en vehículos usados y puede distorsionar el aprendizaje del modelo.

* Registros con años de fabricación muy antiguos (por ejemplo, cercanos a 1900), que no forman parte del mercado moderno de autos usados y no son relevantes para el objetivo de generalización del modelo.

In [ ]:
df = df.query('price>=1000 and price <=40000')
df = df.query('year>=2000')
df = df.query('odometer>=1000 and odometer<=400_000')

Dado que el propósito del proyecto es modelar el comportamiento del mercado actual, estos registros fueron eliminados como primer filtro de outliers, asegurando que el conjunto de datos final sea representativo, coherente y alineado con escenarios reales de predicción.

In [ ]:
df.describe()

,price,year,odometer
count,323189.000000,323189.000000,323189.000000
mean,17118.478321,2012.441918,96883.581762
std,10635.621514,4.979901,61183.275227
min,1000.000000,2000.000000,1000.000000
25%,7988.000000,2009.000000,44339.000000
50%,14999.000000,2013.000000,92720.000000
75%,25039.000000,2017.000000,138259.000000
max,40000.000000,2022.000000,400000.000000


## **Tratamiento de valores nulos variables categoricas**

In [ ]:
df.isnull().mean()

,0
price,0.000000
year,0.000000
manufacturer,0.029373
model,0.007253
condition,0.368008
cylinders,0.408875
fuel,0.007008
odometer,0.000000
title_status,0.017847
transmission,0.004830


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.condition.unique()

array([nan, 'good', 'excellent', 'fair', 'like new', 'new', 'salvage'],
      dtype=object)

In [ ]:
df.condition = df.condition.apply(lambda x: x if x == 'new' 'like new'else x)

Durante el análisis de la variable condición se identificó la presencia de la categoría new, la cual no resulta coherente con el enfoque del dataset, ya que este se centra en autos usados y seminuevos. En el contexto de plataformas de venta de vehículos usados, esta categoría suele utilizarse de forma laxa para describir autos en estado excepcional, pero no necesariamente nuevos de agencia.

Con el objetivo de mantener consistencia semántica y evitar la introducción de ruido categórico, los registros clasificados como new fueron re-etiquetados como like new. Esta categoría representa de forma más precisa vehículos muy bien conservados, con desgaste mínimo y apariencia cercana a la de un auto nuevo.

In [ ]:
df.condition = df.condition.fillna('unknown')

El estado del vehículo (condition) es una de las variables más relevantes para la estimación del precio, ya que refleja directamente el nivel de desgaste, cuidado y uso del automóvil. Por esta razón, no es apropiado eliminar registros con valores faltantes en esta variable, ya que hacerlo implicaría una pérdida significativa de información útil y una reducción innecesaria del tamaño del dataset.

En lugar de eliminar estos registros, se optó por reemplazar los valores nulos por una categoría explícita denominada unknown. Esta estrategia permite:

Conservar la mayor cantidad posible de observaciones

Diferenciar claramente entre un estado conocido y uno no especificado

Permitir que los modelos aprendan si la ausencia de esta información tiene un impacto propio en el precio

Desde el punto de vista del modelado, esta aproximación es especialmente adecuada para algoritmos basados en árboles y técnicas de one-hot encoding, donde las categorías faltantes pueden capturar patrones latentes sin introducir sesgos artificiales.

De esta forma, se mantiene la integridad del conjunto de datos y se preserva el valor predictivo de una variable clave dentro del mercado de autos usados.

In [ ]:
df.cylinders.unique()

array([nan, '8 cylinders', '6 cylinders', '4 cylinders', '5 cylinders',
       '3 cylinders', '10 cylinders', 'other', '12 cylinders'],
      dtype=object)

In [ ]:
df.cylinders = np.where(df.cylinders=="other",np.nan,df.cylinders)

In [ ]:
df.cylinders = df.cylinders.str.replace('cylinders','').astype(float)

Adicionalmente, dentro de la variable cylinders se identificó la presencia de la categoría other, la cual generalmente corresponde a vehículos eléctricos, donde el concepto tradicional de número de cilindros no aplica. Con el fin de mantener coherencia en el tratamiento de la variable, estos registros fueron convertidos a valores nulos, permitiendo un manejo homogéneo durante el proceso de transformación.

In [ ]:
def cyl_bin(x):

    if pd.isna(x):
        return 'unknown'
    elif x <= 4:
        return '4_or_less'
    elif x <= 6:
        return '5_6_cyl'
    else:
        return '8_or_more'


In [ ]:
df.cylinders = df.cylinders.apply(cyl_bin)

In [ ]:
df.cylinders.unique()

array(['unknown', '8_or_more', '5_6_cyl', '4_or_less'], dtype=object)

Posteriormente, y con el objetivo de reducir la cardinalidad de la variable, se realizó una re-agrupación de los valores de cilindros en rangos representativos, basados tanto en criterios mecánicos como en prácticas comunes del mercado automotriz. Esta agrupación permite capturar la relación entre capacidad del motor y precio sin introducir una cantidad excesiva de categorías.

La transformación se realizó mediante la siguiente lógica:

* 4_or_less: Motores de cuatro cilindros o menos, generalmente asociados a vehículos compactos y de enfoque económico.

* 5_6_cyl: Motores de cinco o seis cilindros, comúnmente presentes en versiones de mayor desempeño dentro de un mismo modelo.

* 8_or_more: Motores de ocho cilindros o más, típicos de vehículos de alto desempeño, pickups y modelos orientados al trabajo pesado.

* unknown: Casos donde la información no está disponible o no aplica.

Esta estrategia permite mejorar la interpretabilidad del modelo, reducir ruido categórico y facilitar el aprendizaje tanto en algoritmos lineales como en modelos basados en árboles, manteniendo el significado mecánico de la variable.

In [ ]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,6000,2013.0,NaN,NaN,unknown,unknown,NaN,85548.0,NaN,NaN,NaN,NaN,NaN
1,11900,2013.0,NaN,NaN,unknown,unknown,NaN,85548.0,NaN,NaN,NaN,NaN,NaN
2,21000,2013.0,NaN,NaN,unknown,unknown,NaN,85548.0,NaN,NaN,NaN,NaN,NaN
3,1500,2013.0,NaN,NaN,unknown,unknown,NaN,85548.0,NaN,NaN,NaN,NaN,NaN
4,4900,2013.0,NaN,NaN,unknown,unknown,NaN,85548.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.select_dtypes(include='float').isnull().mean()

,0
year,0.0
odometer,0.0


In [ ]:
nan_values = df.isnull().mean()

In [ ]:
nan_values

,0
price,0.000000
year,0.000000
manufacturer,0.030067
model,0.009467
condition,0.000000
cylinders,0.000000
fuel,0.006943
odometer,0.000000
title_status,0.016432
transmission,0.004586


In [ ]:
def nan_to_mode(x):
  return x.fillna(x.mode()[0])

In [ ]:
columns_obj = df.select_dtypes(include='object').columns
columns_nan_obj = [col for col in columns_obj if nan_values[col] > 0.0 and nan_values[col] < 0.3]

In [ ]:
columns_nan_obj

['manufacturer',
 'model',
 'fuel',
 'title_status',
 'transmission',
 'drive',
 'type',
 'paint_color']

In [ ]:
for column in columns_nan_obj:
  df[column] = df[column].fillna(df[column].mode()[0])

Tras aplicar las transformaciones iniciales y recalcular el porcentaje de valores nulos, se observó que la mayoría de las variables categóricas (tipo object) presentan un porcentaje de datos faltantes inferior al 30%. Este nivel de ausencia se considera manejable sin comprometer la integridad del conjunto de datos.

Dado que estas variables representan categorías discretas, se optó por imputar los valores nulos utilizando la moda, es decir, el valor más frecuente de cada variable. Esta estrategia permite:

* Conservar la distribución original de las categorías.

* Evitar la eliminación innecesaria de registros.

* Mantener la coherencia semántica de las variables.

La imputación por moda es especialmente adecuada en este contexto, ya que las variables categóricas reflejan características comunes del mercado automotriz (como tipo de transmisión, combustible o sistema de tracción), donde los valores dominantes suelen representar configuraciones estándar.

Este enfoque contribuye a reducir la pérdida de información, asegurar consistencia en los datos y facilitar el posterior proceso de codificación categórica previo al entrenamiento de los modelos predictivos.

In [ ]:
 df.isnull().mean()

,0
price,0.0
year,0.0
manufacturer,0.0
model,0.0
condition,0.0
cylinders,0.0
fuel,0.0
odometer,0.0
title_status,0.0
transmission,0.0


In [ ]:
df.drop('model',axis=1,inplace=True)

La variable model contiene información altamente relevante para la estimación del precio, ya que identifica versiones específicas dentro de una misma marca. Por ejemplo, un Nissan Sentra 2014 no tiene el mismo valor de mercado que un BMW M3 del mismo año, aun cuando compartan características como kilometraje similar.

No obstante, durante el análisis se detectaron inconsistencias significativas en los registros, principalmente relacionadas con errores de escritura y variaciones en el formato del nombre del modelo. Casos como "sentra", "s entra" o "centra" representan el mismo vehículo desde el punto de vista del negocio, pero para herramientas como pandas y los codificadores categóricos son interpretados como categorías distintas, lo que incrementa artificialmente la cardinalidad de la variable.

Debido a la alta frecuencia de errores tipográficos, la ausencia de un diccionario confiable de normalización y la dimensión masiva del dataset, se decidió excluir la variable model del proceso de modelado. Mantenerla sin una limpieza exhaustiva introduciría ruido, aumentaría de forma descontrolada el número de categorías y podría afectar negativamente el desempeño y la estabilidad de los modelos.

Esta decisión prioriza la calidad y coherencia de los datos sobre la inclusión de información potencialmente útil pero mal estructurada, permitiendo que los modelos se enfoquen en variables más robustas y consistentes.

## **Guardar dataset**

In [ ]:
df.to_parquet('/content/drive/MyDrive/dataset/vehicles.parquet',index = False)

Dado que el conjunto de datos final está compuesto por 185,680 registros, se optó por almacenar la versión preprocesada en formato Parquet, un formato columnar altamente eficiente tanto en espacio de almacenamiento como en velocidad de lectura y escritura.

El uso de Parquet permite:

* Reducir significativamente el tamaño del archivo en disco gracias a su compresión eficiente.

* Acelerar la carga de datos en análisis posteriores y procesos de modelado.

* Facilitar un flujo de trabajo más escalable y reproducible, especialmente al trabajar con grandes volúmenes de información.

Esta decisión resulta especialmente relevante considerando la complejidad del dataset y la necesidad de reutilizar los datos preprocesados en múltiples notebooks, evitando repetir costosos procesos de limpieza y transformación.

De esta manera, se optimiza tanto el rendimiento computacional como la gestión del proyecto a largo plazo.